<div>
<h1>Template-Based Chord Recognition</h1> 
</div>


A typical chord recognition system consists of two main steps. In the first step, the given audio recording is cut into frames, and each frame is transformed into an appropriate feature vector. Most recognition systems rely on chroma-based audio features, which correlate to the underlying tonal information contained in the audio signal. 


In the second step, pattern matching techniques are used to map each feature vector to a set of predefined chord models. The best fit determines the chord label assigned to the given frame. 

To improve the chord recognition results, additional enhancement techniques are applied either before the pattern matching step (referred to as **prefiltering**) or after/within the pattern matching step 
(referred to as **postfiltering**). In this notebook, we introduce a first chord recognition procedure that employs a simple template-based matching strategy.

<img src="data/img/chord_rec_architecture.png" width="500px" align="middle" alt="chord recognition">

## Formalization of the Chord Recognition Problem

Given an audio recording of a piece of music, the goal of our chord recognition task is to find out which chords are played at which time.  The first step is to transform the recording into a sequence $X=(x_1,x_2,\ldots,x_N)$ of feature vectors $x_n\in\mathcal{F}$, $n\in[1:N]$, where $\mathcal{F}$ denotes a suitable feature space. Then, each feature vector $x_n$ is to be mapped to a chord label $\lambda_{n} \in \Lambda$, where $\Lambda$ denotes a set of possible chord labels. For example, one may consider the set

\begin{equation}
  \Lambda = \{\mathbf{C},\mathbf{C}^\sharp,\ldots,\mathbf{B},\mathbf{Cm},\mathbf{Cm^\sharp},\ldots,\mathbf{Bm}\}
\end{equation}

consisting of the twelve major and minor triads. In this case, each frame $n\in[1:N]$ is assigned to a major chord or a minor chord specified by $\lambda_{n}$. 

For the feature extraction step, basically every chord recognition procedure relies on some type of chroma features. This is because chroma-based features capture a signal's short-time tonal content,
which is closely correlated to the harmonic progression of the underlying piece. Assuming the equal-tempered scale, the chroma values correspond to the set $\{\mathrm{C},\mathrm{C}^\sharp,\mathrm{D},\ldots,\mathrm{B}\}$, which we identify with the set $[0:11]$. A chroma feature can then be expressed as a $12$-dimensional vector $x=(x(0),x(1),\ldots,x(11))^\top\in\mathcal{F}$ with $\mathcal{F}=\mathbb{R}^{12}$.

In the following example, the chord recognition task is illustrated by the first measures of The Beatles' song "Let It Be." As for the chroma representation, a window size that corresponds to $200$ msec and a hop size of half the window length is used. As a result, our feature sequence $X=(x_1,x_2,\ldots,x_N)$ has a feature rate of $10$ Hz. The chord recognition result is obtained in a frame-wise fashion, where subsequent frames with the same chord label are visualized as a single labeled segments.  

<img src="data/img/chord_example.png" width="500px" align="center" alt="example of chord recognition">

<br clear="all" />

<audio style="width: 400px;" src="data/audio/Beatles_LetItBe.wav" type="audio/mpeg" controls="controls"></audio>

## Template-Based Pattern Matching

For the pattern matching step, we now introduce a simple template-based approach. The idea is to precompute a set $\mathcal{T}\subset\mathcal{F}=\mathbb{R}^{12}$ of templates denoted by $\mathbf{t}_\lambda\in\mathcal{T}$, $\lambda\in\Lambda$.
Intuitively, each template can be thought of as a prototypical chroma 
vector that represents a specific musical chord.
Furthermore, we fix a similarity measure 

\begin{equation}
s:\mathcal{F}\times\mathcal{F}\to \mathbb{R}
\end{equation}

that allows for comparing different chroma vectors. Then, the template-based procedure consists in assigning the chord label that maximizes the similarity between the corresponding template and the given feature vector $x_n$:

\begin{equation}
        \lambda_{n} := \underset{\lambda \in \Lambda}{\mathrm{argmax}}
         \,\, s( \mathbf{t}_\lambda , x_n ).
\end{equation}

In this procedure, there are many design choices that crucially influence the performance of a chord recognizer. Which chords should be considered in $\mathcal{T}$? How are the chord templates defined?  What is a suitable similarity measure to compare the feature vectors with the chord templates? 

To obtain a first simple chord recognition system, we make the following design choices. 

For the **chord label set** $\Lambda$, we choose the twelve major and minor triads. This choice, even though problematic from a musical point of view, is convenient and instructive. Considering chords up to enharmonic equivalence and up to octave shifts, each triad can be encoded by a three-element subset of $[0:11]$. For example, the $\mathrm{C}$ major chord $\mathbf{C}$ corresponds tto the subset $\{0,4,7\}$. Each subset, in turn, can be identified with a binary twelve-dimensional chroma vector $x=(x(0),x(1),\ldots,x(11))^\top$, where $x(i)=1$ if and only if the chroma value $i\in[0:11]$ is contained in the chord. For example, in the case of the $\mathrm{C}$ major chord $\mathbf{C}$, the resulting chroma vector is

\begin{equation}
\label{eq:ChordReco:Template:Basic:ChromaVectC}
    \mathbf{t}_{\mathbf{C}}{} := x =(1,0,0,0,1,0,0,1,0,0,0,0)^\top
\end{equation}

Using a chroma-based encoding, the twelve major chords and twelve minor chords can be obtained by cyclically shifting the binary vectors for the $\mathrm{C}$ major and the $\mathrm{C}$ minor triads, respectively. The result is illustrated by the following figure.

<img src="data/img/templates.png" width="800px" align="middle" alt="FMP_C5_F06">




For comparing chroma features and chord templates, we use in the following a simple **similarity measure** using the inner product of normalized vectors:

\begin{equation}
  s(x,y)= \frac{\langle x,y\rangle}{||x||\cdot||y||}
\end{equation}

for $x,y\in\mathcal{F}\setminus\{0\}$. In the case $||x||=0$ or $||y||=0$, we set $s(x,y)=0$. Note that this measure always yields a value $s(x,y)\in[-1,1]$. In the case that the vectors $x$ and $y$ only have positive entries, one has $s(x,y)\in[0,1]$.

In [23]:
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import librosa
import IPython.display as ipd
%matplotlib inline

In [1]:
def generate_template_matrix(templates):
    
    return



## Example


To obtain a better understanding of this procedure, we continue our Beatles example from above. The following steps are performed:

* First, the audio recording is converted into a chroma representation.
* Second, each chroma vector is compared with each of the $24$ binary chord templates, which yields $24$ similarity values per frame. These similarity values are visualized in the form of a **time&ndash;chord representation**.
* Third, we select for each frame the chord label $\lambda_{n}$ of the template that maximizes the similarity value over all $24$ chord templates. This yields our final chord recognition result.

The following figure additionally shows a visualization of the expected (ground-truth) chord labels, which may be used for evaluating our chord recognizer. Furthermore, the similarity-maximizing (normalized) chord templates are shown. In a way, this sequence of chord templates may be thought of as a musically informed quantization of the original chroma representation.

<img src="data/img/letitbe.png" width="600px" align="middle" alt="FMP_C5_F15">

# Feature Filtering
In the previous lecture we have seen of to extract feature from audio recordings (in particular the chromagram).
But feature extraction is not enough and **feature pre or post filtering** can be necessary. We are going to see how to **normalize** and **scale** and how to **smooth** the feautures for enhancing the chord extraction algorithm.

## Feature Normalization

In this application, the audio recording is transformed into a feature representation (chromagram). Often, these representations consists of a sequence $X=(x_1,x_2,\ldots x_N)$ with feature vectors $x_n \in  \mathcal{F}=\mathbb{R}^K$ for $n\in[1:N]$. To better compare feature representations, one often applies **normalization**. One normalization strategy is to choose a suitable norm $p$ and then to replace each feature vector $x_n\in\mathcal{F}$ by $x_n/p(x_n)$. This strategy works as long as $x_n$ is a nonzero vector. Note that the normalized feature vector  $x_n/p(x_n)$ is a unit vector with regard to the norm $p$.

Therefore, let's consider the case that $X=(x_1,x_2,\ldots x_N)$ is a sequence of chroma features. In this case, the feature space $\mathcal{F}=\mathbb{R}^K$ has the dimension $K=12$. The normalization procedure as described above replaces each chroma vector by its normalized version. As a result, a normalized chroma vector only encodes **relative** rather than **absolute** differences in the sizes of the twelve chroma coefficients. Intuitively speaking, normalization introduces a kind of **invariance** to differences in **dynamics** or **sound intensity**.

The normalization procedure is only possible if $p(x)\not= 0$. Also for very small values $p(x)$, which may occur in passages of silence before the actual start of the recording or during long pauses, normalization would lead to more or less random and therefore meaningless chroma value distributions. Therefore, if $p(x)$ falls below a certain threshold, the vector $x$ may be replaced by some standard vector such as a uniform vector of norm one instead of dividing by $p(x)$.

Mathematically, this normalization procedure can be described as follows. Let $S^{K-1}\subset\mathbb{R}^{K}$ be the unit sphere containing all $K$-dimensional vectors of norm one. Then, for a given threshold $\varepsilon>0$, we define a projection operator $\pi^{\varepsilon}:\mathbb{R}^{K}\to S^{K-1}$ by

\begin{equation}
\pi^{\varepsilon}(x):=
\left\{\begin{array}{cl} x / p(x) & \,\,\,\mbox{if}\,\, p(x) > \varepsilon\\
          u & \,\,\,\mbox{if}\,\, p(x) \leq \varepsilon \end{array}\right.
\end{equation}

where $u=v/p(v)$ is the unit vector of the all-one vector $v=(1,1,\ldots,1)^\top\in\mathbb{R}^K$. Based on this operator, each chroma vector $x$ can be replaced by $\pi^{\varepsilon}(x)$. The threshold $\varepsilon$ is a parameter that needs to be chosen with care. A suitable choice will depend on the requirements of the application in mind. One can think of many variants for this normalization. Obviously, the normalization depends on the chosen norm and on the threshold. Furthermore, instead of taking the equally distributed unit vector $u$, one may take any another vector to represent feature vectors of small size. 



## Norms

### Euclidean Norm

The most commonly used norm is the **Euclidean norm** (or $\ell^2$-norm). This norm is often denoted by $\|\cdot\|_2$ and defined by

$$
   \|x\|_2 = \sqrt{\langle x\mid x\rangle} = \Big(\sum_{k=0}^{K-1} x(k)^2\Big)^{1/2}
$$

for a vector $x=(x(0),x(1),\ldots,x(K-1))^\top \in\mathbb{R}^K$.  The Euclidean norm $\|x\|_2$  gives the usual distance from the origin $(0,0)$ to the point $x$. The set of unit vectors with regard to the Euclidean norm forms a **unit sphere** denoted as $S^{K-1}\subset\mathbb{R}^K$. In the case of $K=2$, the unit sphere is the unit circle $S^1$ with origin $(0,0)$. 

### Manhattan Norm

In the **Manhattan norm** (or $\ell^1$-norm), the length of a vector is measured by summing up the absolute values of the vector's Cartesian coordinates. The Manhattan norm, denoted $\|\cdot\|_1$, is defined by 

$$
   \|x\|_1 = \sum_{k=0}^{K-1} |x(k)|
$$

for a vector $x=(x(0),x(1),\ldots,x(K-1))^\top \in\mathbb{R}^K$. The name of the norm comes from the grid-like layout of the streets in Manhattan, which forces a taxi to follow straight lines between the street's intersection points. In the Manhattan norm, the set of unit vectors forms a square with sides oriented at a $45$ degree angle to the coordinate axes. For the case $K=2$, the unit circle of the Manhattan norm is describe by $|x(1)| + |x(2)| = 1$.  

### Maximum Norm

In the **maximum norm** (or $\ell^\infty$-norm), the length of a vector is measured by its maximum absolute Cartesian coordinate. The maximum norm, denoted by $\|\cdot\|_\infty$, is defined by 

$$
   \|x\|_\infty = \max\big\{|x(k)| \,\,\mathrm{for}\,\, k\in[0:K-1] \big\}
$$

for a vector $x=(x(0),x(1),\ldots,x(K-1))^\top \in\mathbb{R}^K$. The set of unit vectors forms the surface of a hypercube with edge length $2$. 



In [25]:
def normalize_feature_sequence(X, norm='2', threshold=0.0001, v=None):
    """Normalizes the columns of a feature sequence

    Args:
        X: Feature sequence
        norm: The norm to be applied. '1', '2', 'max'
        threshold: An+ threshold below which the vector `v` used instead of normalization
        v: Used instead of normalization below `threshold`. If None, uses unit vector for given norm

    Returns:
        X_norm: Normalized feature sequence
    """


    return X_norm

## Feature Smoothing and Downsampling

For certain music retrieval applications, these chromagrams may be too detailed. In particular, it may be desirable to further increase the similarity between them. 

We now show how this can be achieved by **smoothing procedures** applied in a postprocessing step. 
The idea is to compute for each chroma dimension a kind of local average over time. 

More precisely, let $X=(x_1,x_2, ..., x_N)$ be a feature sequence with $x_n\in\mathbb{R}^K$ for $n\in[1:N]$, and let $w$ be a **rectangular window** $w$ of length $L\in\mathbb{N}$. Then we compute for each $k\in[1:K]$ a convolution between $w$ and the sequence $(x_1(k), x_2(k),\ldots, x_N(k))$. Assuming a centered view, we only keep the center part of length $N$ of the convolution. The result is a smoothed feature sequence of the same dimensions $K\times N$.

* In the following implementation, we use the function `scipy.signal.convolve` to compute a 2D convolution. Setting the dimension to $1\times L$ for the **window** (also called **kernel**) results in a bandwise 1D convolution.
* Using the parameter `mode='same'` enforces the centered view.
* As for the window $w$, one may also use other window types such as a **Hann window**. 

Applying temporal smoothing using a rectangular or a Hann window can be regarded as bandwise  **lowpass filtering**, which attenuates fast temporal fluctuations in the feature representation. 

Often, to increase the efficiency of subsequent processing and analysis steps, one decimates the smoothed representation by keeping only every $D^\mathrm{th}$ feature, where $D\in\mathbb{N}$ is a suitable constant (typically much smaller than the window length $L$). This decimation, which is also referred to as **downsampling**, reduces the feature rate by a factor $D$. 


In [3]:
from scipy import signal

def smooth_downsample_feature_sequence(X, Fs, filt_len=41, down_sampling=10, w_type='boxcar'):
    """
    Args:
        X: Feature sequence
        Fs: Frame rate of `X`
        filt_len: Length of smoothing filter
        down_sampling: Downsampling factor
        w_type: Window type of smoothing filter

    Returns:
        X_smooth: Smoothed and downsampled feature sequence
        Fs_feature: Frame rate of `X_smooth`
        
    Hint: use numpy expand dims to obtain a window of dimension (1, L)
    """
    

In this application we are going to use **feature normalization** only, but in the future we are going to need feature smoothing too.

# Template-base chord recognition implementation

As already mentioned, for comparing chroma features and chord templates, we use a simple **similarity measure** using the inner product of normalized vectors:

\begin{equation}
  s(x,y)= \frac{\langle x,y\rangle}{||x||\cdot||y||}
\end{equation}

for $x,y\in\mathcal{F}\setminus\{0\}$. In the case $||x||=0$ or $||y||=0$, we set $s(x,y)=0$. Note that this measure always yields a value $s(x,y)\in[-1,1]$. In the case that the vectors $x$ and $y$ only have positive entries, one has $s(x,y)\in[0,1]$.
